In [1]:
import sys
sys.path.append('..')
sys.path.append('../../Automatic-Circuit-Discovery/')

import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor

from acdc.hybridretrieval.utils import (
    get_all_hybrid_retrieval_things,
    get_gpt2_small
)

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly

import os
import re
import json
import pandas as pd
import torch as t
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.style.use('seaborn-v0_8-paper')


device = t.device("cuda" if t.cuda.is_available() else "CPU")
print(device)

/tmp/ipykernel_253346/306280915.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/tmp/ipykernel_253346/306280915.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')
/home/iustin/.conda/envs/eap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
from acdc.hybridretrieval.utils import (
    get_all_hybrid_retrieval_things,
    get_gpt2_small
)
from acdc.TLACDCExperiment import TLACDCExperiment

all_kbicr_items = get_all_hybrid_retrieval_things(num_examples=20, device=device, metric_name='logit_diff')

model = all_kbicr_items.tl_model
test_metrics = all_kbicr_items.test_metrics['logit_diff']
test_data = all_kbicr_items.test_data
test_patch_data = all_kbicr_items.test_patch_data

exp = TLACDCExperiment(
            model=model,
            threshold=0,
            ds=test_data,
            ref_ds=test_patch_data,
            metric=test_metrics,
            )
exp.model.reset_hooks()
exp.setup_model_hooks(
    add_sender_hooks=True,
    add_receiver_hooks=True,
    doing_acdc_runs=False
)

del all_kbicr_items

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda
Clean Prompts:
Alice lives in France, Paris - Alice, John lives in Germany, Berlin - John, Peter lives in USA, Washington - Peter
Lucy lives in Turkey, Ankara - Lucy, Sara lives in Italy, Rome - Sara, Bob lives in Spain, Madrid - Bob
Tom lives in Canada, Toronto - Tom, Anna lives in Australia, Canberra - Anna, Michael lives in Japan, Tokyo - Michael
David lives in Brazil, Rio de Janeiro - David, Alice lives in France, Paris - Alice, Peter lives in Germany, Berlin - Peter
Sara lives in USA, Washington - Sara, Lucy lives in Turkey, Ankara - Lucy, Tom lives in Italy, Rome - Tom
John lives in Spain, Madrid - John, Michael lives in Canada, Toronto - Michael, Anna lives in Australia, Canberra - Anna
David lives in Japan, Tokyo - David, Sara lives in Brazil, Rio de Janeiro - Sara, Alice lives in France, Paris - Alice
Bob lives in Germany, Berlin - Bob, Peter lives in USA, Washington - Peter, Tom lives in Turkey,

ln_final.hook_normalized
ln_final.hook_scale
blocks.11.hook_resid_post
blocks.11.hook_mlp_out
blocks.11.mlp.hook_post
blocks.11.mlp.hook_pre
blocks.11.ln2.hook_normalized
blocks.11.ln2.hook_scale
blocks.11.hook_mlp_in
blocks.11.hook_resid_mid
blocks.11.hook_attn_out
blocks.11.attn.hook_result
blocks.11.attn.hook_z
blocks.11.attn.hook_pattern
blocks.11.attn.hook_attn_scores
blocks.11.attn.hook_v
blocks.11.attn.hook_k
blocks.11.attn.hook_q
blocks.11.ln1.hook_normalized
blocks.11.ln1.hook_scale
blocks.11.hook_v_input
blocks.11.hook_k_input
blocks.11.hook_q_input
blocks.11.hook_resid_pre
blocks.10.hook_resid_post
blocks.10.hook_mlp_out
blocks.10.mlp.hook_post
blocks.10.mlp.hook_pre
blocks.10.ln2.hook_normalized
blocks.10.ln2.hook_scale
blocks.10.hook_mlp_in
blocks.10.hook_resid_mid
blocks.10.hook_attn_out
blocks.10.attn.hook_result
blocks.10.attn.hook_z
blocks.10.attn.hook_pattern
blocks.10.attn.hook_attn_scores
blocks.10.attn.hook_v
blocks.10.attn.hook_k
blocks.10.attn.hook_q
blocks.10.ln

In [3]:
def clean_pruned_heads(pruned_heads):
    cleaned_heads = {}

    for thresh in pruned_heads.keys():
        cleaned_heads[thresh] = {}
        cleaned_heads[thresh]['acdcpp'] = set()
        cleaned_heads[thresh]['acdc'] = set()

        for i in range(2):
            for head in pruned_heads[thresh][i]:
                attn_head_pttn = re.compile('^<a([0-9]+)\.([0-9]+)>$')
                matched = attn_head_pttn.match(head)
                if matched:
                    head_str = f'{matched.group(1)}.{matched.group(2)}'
                    if i == 0:
                        cleaned_heads[thresh]['acdcpp'].add(head_str)
                    else:
                        cleaned_heads[thresh]['acdc'].add(head_str)
    return cleaned_heads

def get_all_heads(num_layers, num_heads):
    all_heads = set()

    for layer in range(num_layers):
        for head in range(num_heads):
            all_heads.add(f'{layer}.{head}')
    return all_heads

def convert_to_torch_index(index_list):
    return ''.join(['None' if i == ':' else i for i in index_list])

def get_all_edges(exp):
    all_edges = set()
    for child_name in exp.corr.edges:
        for child_idx in exp.corr.edges[child_name]:
            child_idx_converted = convert_to_torch_index(str(child_idx))
            for parent_name in exp.corr.edges[child_name][child_idx]:
                for parent_idx in exp.corr.edges[child_name][child_idx][parent_name]:
                    parent_idx_converted = convert_to_torch_index(str(parent_idx))
                    all_edges.add(f'{parent_name}{parent_idx_converted}{child_name}{child_idx_converted}')
    return all_edges